In [1]:
"""
LSTM text generation by word. Using Asimov's Foundation trilogy as corpus,
the best val_acc I could get is 0.1354 with these parameters:

SEQUENCE_LEN = 10
MIN_WORD_FREQUENCY = 10
STEP = 1
BATCH_SIZE = 32
dropout = 0.4
LSTM(256)

Based on: https://github.com/enriqueav/lstm_lyrics

Uses data generator to avoid loading all the test set into memory.
Saves the weights and model every epoch.
"""


from keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional
import tensorflow as tf
import numpy as np
import sys
import io
import os
import codecs

SEQUENCE_LEN = 10
MIN_WORD_FREQUENCY = 10
STEP = 1
BATCH_SIZE = 32


def shuffle_and_split_training_set(sentences_original, next_original, percentage_test=2):
    # shuffle at unison
    print('Shuffling sentences')

    tmp_sentences = []
    tmp_next_word = []
    for i in np.random.permutation(len(sentences_original)):
        tmp_sentences.append(sentences_original[i])
        tmp_next_word.append(next_original[i])

    cut_index = int(len(sentences_original) * (1.-(percentage_test/100.)))
    x_train, x_test = tmp_sentences[:cut_index], tmp_sentences[cut_index:]
    y_train, y_test = tmp_next_word[:cut_index], tmp_next_word[cut_index:]

    print("Size of training set = %d" % len(x_train))
    print("Size of test set = %d" % len(y_test))
    return (x_train, y_train), (x_test, y_test)


def generator(sentence_list, next_word_list, batch_size):
    index = 0
    while True:
        x = np.zeros((batch_size, SEQUENCE_LEN, len(words)), dtype=np.bool)
        y = np.zeros((batch_size, len(words)), dtype=np.bool)
        for i in range(batch_size):
            for t, w in enumerate(sentence_list[index]):
                x[i, t, word_indices[w]] = 1
            y[i, word_indices[next_word_list[index]]] = 1

            index = index + 1
            if index == len(sentence_list):
                index = 0
        yield x, y

# Functions from keras-team/keras/blob/master/examples/lstm_text_generation.py
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    examples_file.write('\n----- Generating text after Epoch: %d\n' % epoch)

    # Randomly pick a seed sequence
    seed_index = np.random.randint(len(sentences+sentences_test))
    seed = (sentences+sentences_test)[seed_index]

    for diversity in [0.3, 0.4, 0.5, 0.6, 0.7]:
        sentence = seed
        examples_file.write('----- Diversity:' + str(diversity) + '\n')
        examples_file.write(
            '----- Generating with seed:\n"' + ' '.join(sentence) + '"\n')
        examples_file.write(' '.join(sentence))

        for i in range(50):
            x_pred = np.zeros((1, SEQUENCE_LEN, len(words)))
            for t, word in enumerate(sentence):
                x_pred[0, t, word_indices[word]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]

            sentence = sentence[1:]
            sentence.append(next_word)

            examples_file.write(" "+next_word)
        examples_file.write('\n')
    examples_file.write('='*80 + '\n')
    examples_file.flush()

if __name__ == '__main__':

    sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

    # Remove the new lines because they appear randomly
    with io.open('corpus.txt', encoding='utf-8') as f:
        text = f.read().lower().replace('\n', '').replace('’', '\'').replace('"', '')
        print('Corpus length in chars:', len(text))

    # Split corpus into words
    text_in_words = [w for w in text.split(' ') if w.strip() != '']
    print('Corpus length in words:', len(text_in_words))

    word_freq = {}
    for word in text_in_words:
        word_freq[word] = word_freq.get(word, 0) + 1

    ignored_words = set()
    for k, v in word_freq.items():
        if word_freq[k] < MIN_WORD_FREQUENCY:
            ignored_words.add(k)

    words = set(text_in_words)
    print('Unique words before ignoring:', len(words))
    print('Ignoring words with frequency <', MIN_WORD_FREQUENCY)
    words = sorted(set(words) - ignored_words)
    print('Unique words after ignoring:', len(words))

    word_indices = dict((c, i) for i, c in enumerate(words))
    indices_word = dict((i, c) for i, c in enumerate(words))

    # cut the text in semi-redundant sequences of SEQUENCE_LEN words
    sentences = []
    next_words = []
    ignored = 0
    for i in range(0, len(text_in_words) - SEQUENCE_LEN, STEP):
        # Only add sequences where no word is in ignored_words
        if len(set(text_in_words[i: i+SEQUENCE_LEN+1]).intersection(ignored_words)) == 0:
            sentences.append(text_in_words[i: i + SEQUENCE_LEN])
            next_words.append(text_in_words[i + SEQUENCE_LEN])
        else:
            ignored = ignored+1

    print('Ignored sequences:', ignored)
    print('Remaining sequences:', len(sentences))

    (sentences, next_words), (sentences_test, next_words_test) = shuffle_and_split_training_set(sentences, next_words)

    dropout = 0.4
    print('Build model...')
    model = Sequential()
    model.add(Bidirectional(LSTM(256), input_shape=(SEQUENCE_LEN, len(words))))
    if dropout > 0:
        model.add(Dropout(dropout))
    model.add(Dense(len(words)))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

    file_path = "./checkpoints/LSTM_LYRICS-epoch{epoch:03d}-words%d-sequence%d-minfreq%d-" \
                "loss{loss:.4f}-acc{acc:.4f}-val_loss{val_loss:.4f}-val_acc{val_acc:.4f}" % \
                (len(words), SEQUENCE_LEN, MIN_WORD_FREQUENCY)

    checkpoint = ModelCheckpoint(file_path, monitor='val_acc', save_best_only=True)
    print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
    early_stopping = EarlyStopping(monitor='val_acc', patience=5)
    callbacks_list = [checkpoint, print_callback, early_stopping]

    examples_file = open('examples.txt', "w")
    model.fit_generator(generator(sentences, next_words, BATCH_SIZE),
                        steps_per_epoch=int(len(sentences)/BATCH_SIZE) + 1,
                        epochs=100,
                        callbacks=callbacks_list,
                        validation_data=generator(
                            sentences_test, next_words_test, BATCH_SIZE),
                        validation_steps=int(len(sentences_test)/BATCH_SIZE) + 1)


Using TensorFlow backend.


Corpus length in chars: 1198747
Corpus length in words: 212571
Unique words before ignoring: 23624
Ignoring words with frequency < 10
Unique words after ignoring: 2187
Ignored sequences: 193573
Remaining sequences: 18988
Shuffling sentences
Size of training set = 18608
Size of test set = 380
Build model...
Epoch 1/100
582/582 [==============================] - 20s - loss: 6.2259 - acc: 0.0666 - val_loss: 6.1484 - val_acc: 0.0573
Epoch 2/100
582/582 [==============================] - 17s - loss: 5.8007 - acc: 0.0746 - val_loss: 5.9315 - val_acc: 0.0651
Epoch 3/100
582/582 [==============================] - 17s - loss: 5.4671 - acc: 0.0924 - val_loss: 5.8164 - val_acc: 0.0990
Epoch 4/100
582/582 [==============================] - 17s - loss: 5.1137 - acc: 0.1180 - val_loss: 5.7886 - val_acc: 0.1172
Epoch 5/100
582/582 [==============================] - 17s - loss: 4.7295 - acc: 0.1474 - val_loss: 5.8298 - val_acc: 0.1354
Epoch 6/100
582/582 [==============================] - 16s - loss: 

SEQUENCE_LEN = 10
MIN_WORD_FREQUENCY = 10
STEP = 1
BATCH_SIZE = 32
dropout = 0.2, 0.3
LSTM(256)
0.1172 on epoch 6
<br>
SEQUENCE_LEN = 10
MIN_WORD_FREQUENCY = 10
STEP = 1
BATCH_SIZE = 32
dropout = 0.4
LSTM(256)
0.1354 on epoch 6
<br>
SEQUENCE_LEN = 10
MIN_WORD_FREQUENCY = 10
STEP = 1
BATCH_SIZE = 32
dropout = 0.5
LSTM(256)
0.0859 on epoch 6
<br>
SEQUENCE_LEN = 10
MIN_WORD_FREQUENCY = 10
STEP = 1
BATCH_SIZE = 32
dropout = 0.4
LSTM(128)
0.0859 on epoch 6